# Prolog

使用 Prolog 规则生成答案的 LangChain 工具。

## 概述

PrologTool 类允许生成使用 Prolog 规则生成答案的 langchain 工具。

## 设置

让我们在 family.pl 文件中使用以下 Prolog 规则：

parent(john, bianca, mary).\
parent(john, bianca, michael).\
parent(peter, patricia, jennifer).\
partner(X, Y) :- parent(X, Y, _).

In [1]:
#!pip install langchain-prolog

from langchain_prolog import PrologConfig, PrologRunnable, PrologTool

TEST_SCRIPT = "family.pl"

## 实例化

首先创建 Prolog 工具：

In [2]:
schema = PrologRunnable.create_schema("parent", ["men", "women", "child"])
config = PrologConfig(
    rules_file=TEST_SCRIPT,
    query_schema=schema,
)
prolog_tool = PrologTool(
    prolog_config=config,
    name="family_query",
    description="""
        Query family relationships using Prolog.
        parent(X, Y, Z) implies only that Z is a child of X and Y.
        Input can be a query string like 'parent(john, X, Y)' or 'john, X, Y'"
        You have to specify 3 parameters: men, woman, child. Do not use quotes.
    """,
)

## 调用

### 使用带函数调用的 Prolog 工具和 LLM

In [3]:
#!pip install python-dotenv

from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv(), override=True)

#!pip install langchain-openai

from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

要使用该工具，请将其绑定到 LLM 模型：

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools([prolog_tool])

然后查询模型：

In [5]:
query = "Who are John's children?"
messages = [HumanMessage(query)]
response = llm_with_tools.invoke(messages)

LLM 将会响应一个工具调用请求：

In [6]:
messages.append(response)
response.tool_calls[0]

{'name': 'family_query',
 'args': {'men': 'john', 'women': None, 'child': None},
 'id': 'call_gH8rWamYXITrkfvRP2s5pkbF',
 'type': 'tool_call'}

该工具会接收此请求并查询 Prolog 数据库：

In [7]:
tool_msg = prolog_tool.invoke(response.tool_calls[0])

该工具会返回一个列表，其中包含查询的所有解决方案：

In [8]:
messages.append(tool_msg)
tool_msg

ToolMessage(content='[{"Women": "bianca", "Child": "mary"}, {"Women": "bianca", "Child": "michael"}]', name='family_query', tool_call_id='call_gH8rWamYXITrkfvRP2s5pkbF')

然后我们将其传递给 LLM，LLM 利用工具响应来回答原始查询：

In [9]:
answer = llm_with_tools.invoke(messages)
print(answer.content)

John has two children: Mary and Michael, with Bianca as their mother.


## 链式调用

### 将 Prolog 工具与 Agent 结合使用

要将 prolog 工具与代理一起使用，请将其传递给代理的构造函数：

In [10]:
#!pip install langgraph

from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, [prolog_tool])

代理接收查询，并在需要时使用 Prolog 工具：

In [11]:
messages = agent_executor.invoke({"messages": [("human", query)]})

然后代理接收工具的响应并生成答案：

In [12]:
messages["messages"][-1].pretty_print()

================================== Ai Message ==================================

John has two children: Mary and Michael, with Bianca as their mother.


## API 参考

详情请参见 https://langchain-prolog.readthedocs.io/en/latest/modules.html。